# Capstone Project: RAG with PDF Documents

Welcome to this assignment! In this exercise, you'll explore how to perform **Retrieval-Augmented Generation (RAG)** with PDF documents. This format was selected since it is very common but you can do this same process for all sorts of documents. 

## Objectives

By the end of this lab, you will:

- Understand how to use **LangChain** for building RAG pipelines.
- Learn to manage and query document embeddings using **ChromaDB**.
- Build an interactive user interface for querying documents using **Gradio**.

Let’s dive in and uncover how RAG can make document retrieval smarter and more efficient!


#### TIPS FOR SUCCESSFUL GRADING OF YOUR ASSIGNMENT:

- All cells are frozen except for the ones where you need to submit your solutions or when explicitly mentioned you can interact with it.

- You can add new cells to experiment but these will be omitted by the grader, so don't rely on newly created cells to host your solution code, use the provided places for this.
- You can add the comment # grade-up-to-here in any graded cell to signal the grader that it must only evaluate up to that point. This is helpful if you want to check if you are on the right track even if you are not done with the whole assignment. Be sure to remember to delete the comment afterwards!
- Avoid using global variables unless you absolutely have to. The grader tests your code in an isolated environment without running all cells from the top. As a result, global variables may be unavailable when scoring your submission. Global variables that are meant to be used will be defined in UPPERCASE.
- To submit your notebook, save it and then click on the blue submit button at the beginning of the page.

In [ ]:
import os
import glob
import shutil
import gradio as gr

from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from dotenv import load_dotenv, find_dotenv

## Setting Up the Environment

In this step, we ensure that the required environment variables are properly loaded for the lab.

### Key Points:
- The `dotenv` library is used to load environment variables from a `.env` file.
- The code checks if the `OPENAI_API_KEY` is set, which is required for accessing OpenAI's API.
- If the API key is found, a confirmation message is printed.

In [ ]:
_ = load_dotenv(find_dotenv())

if os.getenv("OPENAI_API_KEY"):
	print("OpenAI API key found!")

## Defining Global Variables

This cell sets up important global variables that will be used throughout the lab:

- **`PDF_DOCS_PATH`**: Specifies the path to the directory containing the PDF documents.

- **`EMBEDDINGS`**: Initializes the embeddings model using OpenAI's embeddings.

In [ ]:
PDF_DOCS_PATH = "./docs"

EMBEDDINGS = OpenAIEmbeddings()

## Listing PDF Files

This cell retrieves all PDF files from the specified directory:

- **`glob.glob`**: Searches for files matching the `.pdf` pattern in the directory defined by `PDF_DOCS_PATH`.

- The resulting list, `pdf_files`, contains the paths of all PDF documents found.

The output displays the identified PDF files, which will be processed in subsequent steps.


In [ ]:
# Get all .pdf files in the base directory and its subdirectories
pdf_files = glob.glob(os.path.join(PDF_DOCS_PATH, "*.pdf"))

pdf_files

In this case you will be working with a gardening manual provided by [High Rocks](https://highrocks.org/). However, once you are done with this assignment you can reuse all of this code for your own documents, notice that the code above will locate all PDF files inside the `./docs/` directory which is super useful if you have more than one file.

## Helper Function: `clean_text`

This pre-defined function is provided for your convenience and requires no additional modifications. It preprocesses text to ensure consistent formatting by:

- Replacing newline characters (`\n`) with spaces.
- Removing extra spaces to create clean, well-structured text.

You can directly use this function later in the lab for cleaning text extracted from the PDF documents.

Whenever you are dealing with text you will usually encounter a function similar to this one which will help with the preprocessing to yield better results. This one is pretty basic but these kinds of functions can be done in any way required.


In [ ]:
def clean_text(text: str) -> str:
    # Replace '\n' with spaces and remove multiple spaces
    cleaned = " ".join(text.split("\n"))  # Join on newlines first
    cleaned = " ".join(cleaned.split())  # Remove extra spaces
    return cleaned

## Exercise 1: Implement the `load_pdf` Function

In this exercise, you will write a function to process and clean the content of a PDF document. The function should:

1. Load the PDF file from the specified path.
2. Extract the content of the document, processing each page individually.
3. Use the `clean_text` helper function to ensure the text is properly formatted.

### Hints:
- Utilize [`PyPDFLoader`](https://python.langchain.com/docs/integrations/document_loaders/pypdfloader/) to handle the PDF loading.
- Iterate through the documents to apply text cleaning.
- Return the cleaned content in a format suitable for further processing.

This function will be a foundational part of your pipeline for working with the PDF data. After completing this assignment, if you wish to use this code for your own RAG pipelines you might opt to use a different loader depending on the format of your data. Langchain provides a bunch of loaders which you can check in the [docs](https://python.langchain.com/docs/how_to/#document-loaders) and cover formats such as HTML, JSON and CSV.


In [ ]:
def load_pdf(pdf_path):

	### START CODE HERE ###

	# Use the PyPDFLoader by specifying the correct path
	loader = None

	# Use the load method from the loader to get the documents
	documents = None

	# Iterate over the documents
	for None in None:
		# Apply the clean_text function to the page_content attribute of each document
		None.None = None(None.None)

	### END CODE HERE ###
	
	return documents


Now, the `load_pdf` function will be applied to all the PDF files, combining the cleaned content into a single list. In this case there is a single document so this process is quite fast but it might be slower depending on the amount and size of the files. Notice that the resulting `docs` has a size of 42 which is the same number of pages as the PDF.

In [ ]:
docs = [doc for pdf in pdf_files for doc in load_pdf(pdf)]

print(f"There are a total of {len(docs)} documents")

**Expected Output:**
```
There are a total of 42 documents
```

In [ ]:
# Test your code!
# unittest

## Exercise 2: Implement the `split_documents` function

In this exercise, you will write a function to split the loaded document content into smaller chunks for easier processing. The function should:

1. Use the `RecursiveCharacterTextSplitter` class to split the document content into chunks. You can read more about it [here](https://python.langchain.com/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries).
2. Adjust the chunk size and overlap to ensure that each chunk is appropriately sized while maintaining context between chunks.
3. Return the resulting list of document splits.

### Hints:
- The `RecursiveCharacterTextSplitter` class allows you to configure how the text is split. You’ll need to adjust the following parameters:

  - **`chunk_size`**: Set the maximum size of each chunk (use 1500 characters for this exercise).

  - **`chunk_overlap`**: Define the number of characters to overlap between chunks (use 150 characters for this exercise).
  - **`separators`**: Specify the separators that should be used to break the text into chunks. These can include sentence-ending punctuation or newlines. These are already provided.
  - **`keep_separator`**: Set to `True` to retain the separator in the split text.

- To perform the actual splitting you need to use the [`split_documents`](https://python.langchain.com/api_reference/text_splitters/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html#langchain_text_splitters.character.RecursiveCharacterTextSplitter.split_documents) method from `RecursiveCharacterTextSplitter`.


This function will help break down long documents into manageable pieces that are suitable for embedding and processing.


In [ ]:
def split_docs(docs):

	### START CODE HERE ###

	# Instantiate the RecursiveCharacterTextSplitter class with the appropriate parameters
	text_splitter = RecursiveCharacterTextSplitter( 
		chunk_size=None,
		chunk_overlap=None,
		separators=[". ", "? ", "! ", "\n\n", "\n", " ", ""], 
		keep_separator=None,
	)

	# Use the splitter to split the documents (use the split_documents method)
	splits = None

	### END CODE HERE ###

	return splits

Now apply the splitting function to the actual documents:

In [ ]:
splitted_docs = split_docs(docs)

print(f"There are a total of {len(splitted_docs)} documents after splitting")

**Expected Output:**
```
There are a total of 52 documents after splitting
```

In [ ]:
# Test your code!
# unittest

## Exercise 3: Implement the `setup_vectordb` function

In this exercise, you will write a function to set up a vector database for efficient document retrieval. The function should:

1. Use the `Chroma.from_documents` method to create a vector database from the split documents.
2. Embed the documents using the specified embeddings model.
3. Store the vector database in the specified directory for persistent storage.

### Hints:

- The method accepts three parameters:

  - **`splits`**: The list of document splits to be embedded.
  
  - **`embeddings`**: The embeddings model used to convert documents into vectors. Here you can use the global variable for embeddings defined earlier.
  - **`db_docs_path`**: The directory where the vector database will be stored. The default is `db/chroma/`. Use the provided parameter `db_docs_path` to ensure a proper grading.


This function sets up the vector database that will be used for document retrieval in subsequent steps.


In [ ]:
def setup_vectordb(splitted_docs, db_docs_path="db/chroma/"):

	# Delete the in-memory directory that will hold the data
	# This is done in case you run this function multiple times to avoid duplicated documents
	if os.path.exists(db_docs_path) and os.path.isdir(db_docs_path):
		shutil.rmtree(db_docs_path)

	### START CODE HERE ###

	# Create an instance of the vector database
	vectordb = Chroma.from_documents( 
		documents=None,
		embedding=None,
		persist_directory=None,
	)

	### END CODE HERE ###

	return vectordb

Now run the function to create the vector database that contains the post-splitting documents:

In [ ]:
DATABASE = setup_vectordb(splitted_docs)

if os.path.exists("./db/chroma/"):
	print("Successfully created the vector database!")
else:
	print("The directory to store the vector database was not created, double check your code.")

**Expected Output:**
```
Successfully created the vector database!
```

In [ ]:
# Test your code!
# unittest

Now try asking the database to retrieve the top k (5 in this case) documents given a question. You might get duplicated results but that is ok!

You can try out different questions and values of k.

In [ ]:
question = "How can I plant tomatoes?"
retrieved_docs = DATABASE.similarity_search(question, k=5)

for rd in retrieved_docs:
	print(rd)

## Helper Function: `format_docs`

To be able to visualize the retrieved documents in a more organized manner, the `format_docs` function is provided. This function prints out some information about the documents such as their number, contents, document of origin and page:


In [ ]:
def format_docs(docs):
    results = []
    for i, doc in enumerate(docs, 1):
        source_path = doc.metadata.get("source", "Unknown")
        filename = (
            os.path.basename(source_path) if source_path != "Unknown" else "Unknown"
        )

        result = (
            f"Result {i}:\n"
            f"{doc.page_content}\n\n"
            f"Document: {filename}\n"
            f"Page: {doc.metadata.get('page', 'Unknown')}"
        )
        results.append(result)

    return "\n---\n".join(results)

Try it out with the retrieved documents:

In [ ]:
print(format_docs(retrieved_docs))

## Exercise 4: Implement the `process_query` function

In this exercise, you will complete the logic for handling user queries in a retrieval-augmented generation (RAG) pipeline. Notice that the first two steps have already been provided for you:

1. Initialize a language model (LLM) for generating responses.

2. Define a custom prompt template to ensure detailed, thorough answers.


### Hints:
- Use **`PromptTemplate.from_template`** to convert the provided template into a prompt object.
- Retrieve the source documents:
  - Use the **`as_retriever`** method on `DATABASE` to set up a retriever.
  - Invoke the retriever with the given question to get the relevant documents.
  - Use the `format_docs` helper function to process and format the retrieved documents.
- Set up the QA pipeline:
  - Define a chain where:
    - **Context** is set to the retriever output.
    - **Question** is directly passed through with `RunnablePassthrough`.
    - **Prompt** is piped into the QA chain.
    - **LLM** processes the prompt.
    - **StrOutputParser** is used to parse the LLM's output.
  - Use the `invoke` method on the QA chain to generate the LLM's response.

This task will test your ability to integrate multiple components in the RAG pipeline effectively. For more info be sure to check the [docs](# https://python.langchain.com/docs/how_to/inspect/).



In [ ]:
def process_query(question):
	
	# Initialize the LLM
	llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

	# Define a template for the QA
	template = """Use the following pieces of context to answer the question at the end. Provide a detailed, thorough response that:
		1. Answers the main question
		2. Provides relevant examples or details from the context
		3. Explains any important concepts mentioned
		4. If relevant, discusses implications or applications

		If you don't know the answer, provide a detailed explanation of what aspects you're uncertain about and why.

		{context}
		Question: {question}
		Detailed Answer:"""

	### START CODE HERE ###

	# Instantiate a PromptTemplate using the template given
	prompt = None

	# Use the as_retriever method to use the DATABASE as a retriever
	retriever = None
	
	# Get the source documents by using the invoke method on the retriever and passing the question
	source_documents = None

	# Format the source documents using the format_docs helper function
	doc_references = None

	# Set up the QA chain
	qa_chain = ( 
	# Use the retriever as context and a RunnablePassthrough as question
	{"context": None, "question": None}
	# Pipe to the prompt
	| None
	# Pipe to the llm
	| None
	# Pipe to the StrOutputParser
	| None
	) 

	# Get response from qa_chain by using the invoke method and passing the question
	llm_response = None

	### END CODE HERE ###

	return llm_response, doc_references

In [ ]:
question = "How can I plant flowers?"

llm_response, doc_references = process_query(question)

print(f"### LLM Response #################\n\n{llm_response}\n")
print(f"### References ###################\n\n{doc_references}")

In [ ]:
# Test your code!
# unittest

## Gradio User Interface

This cell creates an interactive Gradio interface for you to interact with the Q&A Assistant. The interface allows users to ask questions and receive detailed answers along with source document references.

### Key Features:
- **Input Section**: A textbox for entering questions, with a "Submit Question" button for submission.
- **Tabs for Output**:
  - **AI Response**: Displays the detailed answer generated by the AI.
  - **Document References**: Shows the source documents referenced in the response.
- **Interactivity**:
  - The "Submit Question" button triggers the `process_query` function to process the input and display the results.
  - Pressing "Enter" in the question textbox also submits the question.

This user-friendly interface enables interaction with the QA chatbot in a structured and visually organized way.


In [ ]:
with gr.Blocks(theme=gr.themes.Monochrome()) as iface:
    gr.Markdown(
        """
        # 📚 DAG Scripts Q&A Assistant
        Ask any question about the DAG C1 & C2 content and get detailed answers with source references.
        """
    )

    with gr.Column():
        gr.Markdown("### Your Question")
        question_input = gr.Textbox(
            lines=3,
            placeholder="Enter your question here...",
            label="",  # Removed the label since we're using Markdown
        )
        submit_btn = gr.Button("Submit Question", variant="primary", size="lg")

    with gr.Tabs():
        with gr.TabItem("📝 Response"):
            gr.Markdown("### AI Response")
            response_output = gr.Textbox(
                lines=15,
                label="",  # Removed the label since we're using Markdown
                show_copy_button=True,
            )
        with gr.TabItem("🔍 Document References"):
            gr.Markdown("### Source Documents")
            references_output = gr.Textbox(
                lines=15,
                label="",  # Removed the label since we're using Markdown
                show_copy_button=True,
            )

    # Add submit button click event and enter key functionality
    submit_btn.click(
        fn=process_query,
        inputs=[question_input],
        outputs=[response_output, references_output],
    )
    question_input.submit(
        fn=process_query,
        inputs=[question_input],
        outputs=[response_output, references_output],
    )

In [ ]:
# Close the server (in case you run this cell multiple times)
iface.close()

# Spin up the gradio app
iface.launch(server_name="0.0.0.0", share=False)

## 🎉 Congratulations!

You’ve successfully completed the assignment! 

In this lab, you:
- Processed and cleaned PDF documents for use in a retrieval-augmented generation (RAG) pipeline.
- Split documents into manageable chunks for embedding.
- Set up a vector database for efficient retrieval.
- Implemented a QA pipeline to handle user queries with detailed responses and source references.
- Built an interactive Gradio interface to bring the system to life.

These skills are essential for working with AI-powered applications that combine large language models with domain-specific knowledge. Great work!

Keep experimenting and exploring—there’s so much more you can build with these tools. 🚀
